In [1]:
from trailmet.algorithms.distill.factor_transfer import FactorTransfer, Paraphraser
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

# Dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Pad(4, padding_mode='reflect'),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomCrop(32),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transform_train_paraphraser = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transforms1 = {
    'train': transform_train, 
    'val': transform_test, 
    'test': transform_test}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

In [4]:
from trailmet.datasets.classification import DatasetFactory

cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = "./data",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.1,
                                        transform = transforms1,
                                        target_transform = target_transforms,
                                        random_seed=27
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=128, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=2
    )

val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=128, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=2
    )

test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=128, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=2
    )

In [6]:
dataloaders = {'train':train_loader, 'val':val_loader, 'test':test_loader}

In [7]:
transforms1 = {
    'train': transform_train_paraphraser, 
    'val': transform_test, 
    'test': transform_test}

In [8]:
cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = "./data",
                                        split_types = ['train', 'val'],
                                        val_fraction = 0.1,
                                        transform = transforms1,
                                        target_transform = target_transforms,
                                        random_seed=27
                                        )

Files already downloaded and verified
Files already downloaded and verified


In [9]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=128, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=2
    )

val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=128, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=2
    )

In [10]:
#dataloaders['paraphraser'] = {'train':train_loader, 'val':val_loader}

# Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()



        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        # 1/2 width & height
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

        # 0 - layer1 ... 4 - output
    def forward(self, x):
        #attention_map=[];
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        #attention_map.append(out1)
        out2 = self.layer2(out1)
        #attention_map.append(out2)
        out3 = self.layer3(out2)
        #attention_map.append(out3)
        out4 = self.layer4(out3)
        #attention_map.append(out4)
        out = F.avg_pool2d(out4, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        #attention_map.append(out)
        return out

class ResNet_simple(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_simple, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        # 1/2 width & height
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(64*block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

        # 0 - layer1 ... 3 - output
    def forward(self, x):
        #attention_map=[]
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        #attention_map.append(out1)
        out2 = self.layer2(out1)
        #attention_map.append(out2)
        out3 = self.layer3(out2)
        #attention_map.append(out3)
        out = F.avg_pool2d(out3, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        #attention_map.append(out)
        return out


def ResNet20():
    return ResNet_simple(BasicBlock, [3,3,3])

def ResNet56():
    return ResNet_simple(BasicBlock, [9,9,9])

def ResNet112():
    return ResNet_simple(BasicBlock, [18,18,18])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


In [8]:
teacher_model = ResNet56()
student_model = ResNet20()

teacher_model.to(device)
student_model.to(device)

ResNet_simple(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, 

In [9]:
teacher_model.load_state_dict(torch.load('../FactorTransfer/teacher2.pt'))
#teacher_model.load_state_dict(torch.load('./Teacher.pth')['state_dict'])

<All keys matched successfully>

In [10]:
kwargs = {
    'DISTILL_ARGS' : {
        'BETA':500,
        'EPOCHS':163,
        'LR':0.1,
        'WEIGHT_DECAY':0.0005,
        
        'IN_PLANES':64,
        'RATE':0.5,
        
        'TEACHER_LAYER_NAME':'layer3',
        'STUDENT_LAYER_NAME':'layer3',
    },

    'PARAPHRASER' : {
        'IN_PLANES':64,
        'RATE':0.5,
        'EPOCHS':5,
        'LR':0.1,
        'WEIGHT_DECAY':0.0005,
    },
    
    'DEVICE' : device,
    'log_dir' : 'ft_resnet20'
}

In [11]:
box = FactorTransfer(teacher_model, student_model, dataloaders, **kwargs)

In [12]:
box.compress_model()

=====TRAINING PARAPHRASER=====


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 352/352 [00:24<00:00, 14.41it/s, loss=0.375]


Epoch 1 | Train loss: 0.3755


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 352/352 [00:22<00:00, 15.82it/s, loss=0.206]


Epoch 2 | Train loss: 0.2059


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 27.40it/s, acc=0.958, loss=2.06]


Epoch: 162


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 29.70it/s, acc=0.959, loss=2.07]


Epoch: 163


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 32.11it/s, acc=0.958, loss=2.07]


In [15]:
torch.save(student_model.state_dict(), "./checkpoints/ft_resnet20_last.pth")

In [18]:
student_model.load_state_dict(torch.load('./checkpoints/ft_resnet20.pth')['state_dict'])

<All keys matched successfully>

In [19]:
preds = []
valid_labels = []
student_model.eval()
# Run the best model on test set
for step, (images, labels) in tqdm(enumerate(test_loader), total = len(test_loader)):

    images = images.to(device, dtype=torch.float)
    labels = labels.to(device)
    batch_size = labels.size(0)

    with torch.no_grad():
        y_preds = student_model(images)
        
    preds.append(y_preds.softmax(1).to('cpu').numpy())
    valid_labels.append(labels.to('cpu').numpy())

predictions = np.concatenate(preds)
valid_labels = np.concatenate(valid_labels)

# Get the score
score = accuracy_score(valid_labels, predictions.argmax(1))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 52.73it/s]


In [20]:
score

0.931